In [1]:
from astropy.io import fits

In [3]:
hdus = fits.open("../tests/data/WFPC2u57.fits")

In [7]:
bytes = open("../tests/data/WFPC2u57.fits", "rb").read()

In [29]:
bytes[(2880 * 8) + 480000:]

b'?\x8c\x0c\xf0?dB\x08\xbfm\xbd\xd4\xbe@\xf26?\n\x90\x19\xbf\x85Z\x8d?\x909$\xbfqi\xec\xbf\x87s\x82\xbc\xa6\x0f\xe0\xbf~\xe6H\xbd\xaf\x83\x00=\xb5\x1d\x88?c\xaeI?eE\r?E\x08\x10?\x80\xff\xe7\xbf\x8f\xf2\xa0\xbf\x86\x10Z\xbf~\xd6\xde?Y\x18<?~X??\x81\xe6\x8f\xbev\t\x02<\x85\x8a\x00\xbd\xd0\n\x06\xbc\x8d\x018?H;\xcf?L\x9d\\\xbe;>\xdf>\x14d\x9c?_\x95D?\xfeY\xcb\xbd\xb7Q\n\xbc\xfd \x10\xbe\x88L\x9e=\x1e\x82x\xbe\x17\x82\xcf:\xcdA\x00?\x0e\xcc\xaf\xbd_xp\xbd\xe0\x00\xd2<\xf5\xc7\x18?;\xd2t?S6\x02\xbc\x00\xce ?Pm\x80\xbe\x90J\x9d\xbb\xcf\x86`\xbe\x11=}\xbfsk\xc4\xbf\x8e\r~\xbd\xd3\x84\x1a\xbd\xd9.\xc2\xbeM\x14#\xbd+\xda\x98\xbc\xf9~\xc0?S\xd5 @\x02\x9f\xfa?)\xd9\x01\xbd\xa9\x13d\xbd\x9c\xb9l?\x90Q\xc8?/3\xde\xbf{U\xa0?h\xf5\xc1=\x98=p;\xaac\x90\xbc\xd9\xca\xb0\xbe\x9c\xfd%\xbf\x84\t\x83\xbf\x87\'\n?\xf4\xb3\xde?\xe7\xa7t\xbc\x1bI\xa0\xbeW\xb0\xba?\x82\xe5\n\xbd\xd5~\xc6\xbd\xe8\xe6~\xbe\x04\x1f ?H\xd0\x18\xbel\xa4\xc8\xbf\x84%\x0c\xbet\x83\x9e?\xf0\xb80\xbd\xef\x1d\xb6\xbf\x88\x0e\x80\xbcx\xc8

In [30]:
(2880 * 8) + 480000

503040

In [52]:
def get_first_header_size(bytes):
    # Header may be multiple of 2880 bytes, get the size of the header until the block where the END card is present
    header_size = 0
    while True:
        header_size += 2880
        if b"END     " in bytes[header_size:header_size + 2880]:
            header_size += 2880
            break
        
    return header_size


In [73]:
bytes[23040 + 642240:]

b'XTENSION= \'TABLE   \'           / Ascii table extension                          BITPIX  =                    8 / 8-bits per \'pixels\'                            NAXIS   =                    2 / Simple 2-D matrix                              NAXIS1  =                  796 / Number of characters per row                   NAXIS2  =                    4 / The number of rows                             PCOUNT  =                    0 / No \'random\' parameters                         GCOUNT  =                    1 / Only one group                                 TFIELDS =                   49 / Number of fields per row                       EXTNAME = \'u5780205r_cvt.c0h.tab\' / Name of table                                                                                                               TTYPE1  = \'CRVAL1  \'           /                                                TBCOL1  =                    1 /                                                TFORM1  = \'D25.17  \'      

In [76]:
hdus[1].header

XTENSION= 'TABLE   '           / Ascii table extension                          
BITPIX  =                    8 / 8-bits per 'pixels'                            
NAXIS   =                    2 / Simple 2-D matrix                              
NAXIS1  =                  796 / Number of characters per row                   
NAXIS2  =                    4 / The number of rows                             
PCOUNT  =                    0 / No 'random' parameters                         
GCOUNT  =                    1 / Only one group                                 
TFIELDS =                   49 / Number of fields per row                       
EXTNAME = 'u5780205r_cvt.c0h.tab' / Name of table                               
                                                                                
TTYPE1  = 'CRVAL1  '           /                                                
TBCOL1  =                    1 /                                                
TFORM1  = 'D25.17  '        

In [53]:
get_first_header_size(bytes)

23040

In [71]:
def get_size_in_bytes(header):
    ## It may have multiple NAXISn keywords, so we need to get the product of all of them
    size_in_bytes = 1
    for i in range(1, header["NAXIS"] + 1):
        print(header["NAXIS" + str(i)])
        size_in_bytes *= header["NAXIS" + str(i)]
    
    size_in_bytes *= abs(header["BITPIX"] / 8)
    ## Make it multiple of 2880 bytes
    if size_in_bytes % 2880 != 0:
        size_in_bytes += 2880 - (size_in_bytes % 2880)
    
    return size_in_bytes
    